In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import sys
import time
import torch

from scipy.integrate import solve_ivp

PROJECT_ROOT = os.path.abspath(
    os.path.join(os.getcwd(), os.pardir)
)
sys.path.append(PROJECT_ROOT)

import sample_points
from tcpinn import TcPINN
from plot import plot_solution

np.random.seed(1)

In this example, we solve a Lotka-Volterra Equation of the general form

\begin{align*}
    \frac{dr}{dt} = \alpha r - \beta rp \\
    \frac{dp}{dt} = \delta rp - \gamma p
\end{align*}

where $r$ is the number of prey, $p$ is the number of some predator, and $\alpha, \beta, \gamma, \delta > 0$ describe the interactions of the two species.

In [2]:
alpha = 1.
beta = 1.
gamma = 1.
delta = 1.

In [3]:
class LotkaVolterra(TcPINN):
    """
    A tcPINN implementation of the Lotka-Volterra equations.
    """
    def __init__(
        self, layers, T, X_pinn=None, X_semigroup=None, X_smooth=None, X_data=None, data=None,
        w_pinn=1., w_semigroup=1., w_smooth=1., w_data=1.
    ):
        super().__init__(
            layers, T, X_pinn, X_semigroup, X_smooth, X_data, data,
            w_pinn, w_semigroup, w_smooth, w_data
        )

    
    def _loss_pinn(self):
        """
        Lotka-Volterra equations:
            dr/dt = alpha * r - beta * r * p
            dp/dt = delta * r * p - gamma * p
        """
        y = self.net_y(self.t_pinn, self.y_pinn)
        deriv = self.net_derivative(self.t_pinn, self.y_pinn)

        loss1 = torch.mean(
            (deriv[0] - alpha * y[:, 0:1] + beta * y[:, 0:1] * y[:, 1:2]) ** 2
        )
        loss2 = torch.mean(
            (deriv[1] - delta * y[:, 0:1] * y[:, 1:2] + gamma * y[:, 1:2]) ** 2
        )
        loss = self.w_pinn * (loss1 + loss2)

        return loss

### Setup data example

In [4]:
ode_dimension = 2
layers = [ode_dimension + 1] + 6 * [64] + [ode_dimension]

T = 1
max_y0 = 5

# training samples
n_pinn = 10
t_pinn = np.random.uniform(0, T, (n_pinn, 1))
y_pinn = np.random.uniform(0, max_y0, (n_pinn, ode_dimension))
X_pinn = np.hstack([t_pinn, y_pinn])

n_semigroup = 10
st_semigroup = sample_points.uniform_triangle_2d(n_semigroup, T)
y_semigroup = np.random.uniform(0, max_y0, (n_semigroup, ode_dimension))
X_semigroup = np.hstack([st_semigroup, y_semigroup])

n_smooth = 10
t_smooth = np.random.uniform(0, T, (n_smooth, 1))
y_smooth = np.random.uniform(0, max_y0, (n_smooth, ode_dimension))
X_smooth = np.hstack([t_smooth, y_smooth])

In [5]:
model = LotkaVolterra(layers, T, X_pinn=X_pinn, X_semigroup=X_semigroup, X_smooth=X_smooth)

In [6]:
%%time               
model.train()

CPU times: user 1.54 s, sys: 1.23 ms, total: 1.54 s
Wall time: 234 ms


In [7]:
path = os.getcwd()

with open(f"{path}/model_lotka_volterra.pkl", "wb") as handle:
    pickle.dump(model, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(f"{path}/model_lotka_volterra.pkl", "rb") as f:
    model = pickle.load(f)

## Predict and Plot the Solution

In [9]:
def rhs_lotka_volterra(t, y, alpha, beta, gamma, delta):
    
    r, p = y
    dr_t = alpha * r - beta * r * p
    dp_t = delta * r * p - gamma * p
    
    return dr_t, dp_t


def get_solution(max_t, delta_t, y0):
    
    times = np.linspace(0, max_t, int(max_t / delta_t) + 1)
    sol = solve_ivp(
        rhs_lotka_volterra, [0, float(max_t)], y0, t_eval=times,
        rtol=1e-10, atol=1e-10, args=(alpha, beta, gamma, delta)
    )
    return sol.y.T

In [13]:
y0 = np.array([0.15, 1])
max_t = 40
delta_t = 0.05
times = np.linspace(0, max_t, int(max_t / delta_t) + 1)

true_solution = get_solution(max_t, delta_t, y0)
tc_solution = model.predict_tc(max_t, delta_t, y0)

ax = plot_solution(
    times, true_solution, 
    component_kwargs=[{'color': "black", 'label': "truth"}, {'color': "black"}]
)
ax = plot_solution(
    times, tc_solution, ax=ax,
    component_kwargs=[{'color': "blue", 'label': "tcPINN"}, {'color': "blue"}]
)
plt.legend()
plt.show()
plt.close()